## Preparations

We import `pymovements` as the alias `pm` for convenience.

Let's start by downloading a toy dataset `ToyDatasetEyeLink` that contains `*.asc` files:

This dataset includes `*.asc` files that store raw eye-tracking data along with synchronization messages. Below, we’ll inspect the files included in the dataset:

Let’s display the first 20 lines of one of the files to get a sense of its structure:

We can see that this file is a converted version of an `*.edf` file created by EyeLink.

Let’s try loading one of these files directly using `pm.gaze.from_asc`:

In [ ]:
gaze = pm.gaze.from_asc(file=asc_files[0])
gaze

In [ ]:
gaze.samples

In [ ]:
gaze.experiment

### Loading eye-tracking data along with SR Research recording messages
To extract all `MSG`-prefixed SR Research messages, simply pass `True` to the `pm.gaze.from_asc`. The messages are stored in `gaze.messages`:

We can also control which messages are parsed by specifying them in the `messages` argument. For example, to extract only trial-related messages containing the keyword `TRIAL`, we can do the following:

### Defining custom patterns for data extraction

Now let’s define our own patterns to extract additional information from the `*.asc` files and add them to the `GazeDataFrame`.
We can do this using the parameter `patterns` using `pm.gaze.from_asc`.

`patterns` accepts either a list of custom patterns to match additional columns or a key identifying predefined and eye-tracker-specific patterns.

Let’s define a set of custom patterns to extract more information from parsed messages and show the resulting `GazeDataFrame`:

The examples above illustrate that patterns can be defined in different forms. Some patterns simply match a message and assign a fixed column value (see the first pattern above), while others use regular expressions to capture dynamic information—for instance, the `trial_id` in the last pattern.

Given the patterns defined above, we can see that the columns for `task` and `trial_id` has been added.

The `trial_id` was extracted from messages such as `MSG 2762689 TRIALID 0`, while the task value was obtained from messages like `MSG 2814942 SYNCTIME_JUDO`.

In [ ]:
resource_definition = pm.ResourceDefinition(
    content='gaze',
    filename_pattern=r'subject_{subject_id:d}_session_{session_id:d}.asc',
    filename_pattern_schema_overrides={
        'subject_id': int,
        'session_id': int,
    },
    load_kwargs={
        'patterns': patterns,
        'schema': {'trial_id': int},
    },
)

In [ ]:
experiment = pm.Experiment(
    screen_width_px=1280,
    screen_height_px=1024,
    screen_width_cm=38,
    screen_height_cm=30.2,
    distance_cm=68,
    origin='lower left',
    sampling_rate=1000,
)

In [ ]:
dataset_definition = pm.DatasetDefinition(
    name='ToyDatasetEyeLink',
    experiment=experiment,
    resources=[resource_definition],
)

In [ ]:
dataset = pm.Dataset(
    definition=dataset_definition,
    path='data/ToyDatasetEyeLink',
)
dataset.load()

In [ ]:
dataset.gaze[0].samples